# Synthetic Data Generation: Libraries Comparison

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sdv.single_table import GaussianCopulaSynthesizer
from ydata.sdk.dataset import get_dataset
from ydata.sdk.synthesizers import RegularSynthesizer

print("Hello")

Hello
